# Import packages

In [31]:
import torch
import sys
import numpy as np
import matplotlib.pyplot as plt
import torchvision

from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets

# Configuration

In [32]:
device = 'cuda:3' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

arch = 'resnet50'
dataset_name = 'cifar10'

if arch == 'resnet18':
  model = torchvision.models.resnet18(pretrained=False, num_classes=10).to(device)
elif arch == 'resnet50':
  model = torchvision.models.resnet50(pretrained=False, num_classes=10).to(device)

epochs = 200
batch_size = 256
dataset = 'cifar10'
lr = 1e-3
w = 0

num_workers = 8

Using device: cuda:3


# Prepare DataLoaders

In [33]:
def get_stl10_data_loaders(download, shuffle=True, batch_size=batch_size):
  train_dataset = datasets.STL10('../dataset', split='train', download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=num_workers, drop_last=True, shuffle=shuffle)
  
  test_dataset = datasets.STL10('../dataset', split='test', download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=num_workers, drop_last=True, shuffle=shuffle)
  return train_loader, test_loader

def get_cifar10_data_loaders(download, shuffle=True, batch_size=batch_size):
  train_dataset = datasets.CIFAR10('../dataset', train=True, download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=num_workers, drop_last=True, shuffle=shuffle)
  
  test_dataset = datasets.CIFAR10('../dataset', train=False, download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=num_workers, drop_last=True, shuffle=shuffle)
                            
  return train_loader, test_loader

# Modify SimCLR's checkpoint state_dict to fit ResNet Classification Task.

In [34]:
checkpoint = torch.load('../result/checkpoint/simclr/checkpoint_0100.pth.tar', map_location=device)
state_dict = checkpoint['state_dict']
for k in list(state_dict.keys()):
  if k.startswith('backbone.'):
    if k.startswith('backbone') and not k.startswith('backbone.fc'):
      # remove prefix
      state_dict[k[len("backbone."):]] = state_dict[k]
  del state_dict[k]

In [35]:
log = model.load_state_dict(state_dict, strict=False)
assert log.missing_keys == ['fc.weight', 'fc.bias']

In [36]:
if dataset_name == 'cifar10':
  train_loader, test_loader = get_cifar10_data_loaders(download=True)
elif dataset_name == 'stl10':
  train_loader, test_loader = get_stl10_data_loaders(download=True)
print("Dataset:", dataset_name)

Files already downloaded and verified
Files already downloaded and verified
Dataset: cifar10


# Frozen ResNet parameter and train a classifier on its hidden representation to eval SimCLR's performance

In [37]:
# freeze all layers but the last fc
for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        param.requires_grad = False

# filter(function, iterable)
parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
assert len(parameters) == 2  # fc.weight, fc.bias

In [38]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=w)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [39]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [40]:
for epoch in range(epochs):

  model.train()

  top1_train_accuracy = 0
  
  for counter, (x_batch, y_batch) in enumerate(train_loader):

    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
    loss = criterion(logits, y_batch)
    top1 = accuracy(logits, y_batch, topk=(1,))
    top1_train_accuracy += top1[0]

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  top1_train_accuracy /= (counter + 1)

  top1_accuracy = 0
  top5_accuracy = 0

  model.eval()
  with torch.no_grad():
    for counter, (x_batch, y_batch) in enumerate(test_loader):

      x_batch = x_batch.to(device)
      y_batch = y_batch.to(device)

      logits = model(x_batch)
    
      top1, top5 = accuracy(logits, y_batch, topk=(1,5))
      top1_accuracy += top1[0]
      top5_accuracy += top5[0]
    
  top1_accuracy /= (counter + 1)
  top5_accuracy /= (counter + 1)
  print(f"Epoch {epoch}\tTop1 Train accuracy {top1_train_accuracy.item()}\tTop1 Test accuracy: {top1_accuracy.item()}\tTop5 test acc: {top5_accuracy.item()}")

Epoch 0	Top1 Train accuracy 19.46915054321289	Top1 Test accuracy: 20.764802932739258	Top5 test acc: 70.09662628173828
Epoch 1	Top1 Train accuracy 22.23357391357422	Top1 Test accuracy: 22.101152420043945	Top5 test acc: 70.74424743652344
Epoch 2	Top1 Train accuracy 23.63381576538086	Top1 Test accuracy: 22.1217098236084	Top5 test acc: 70.1274642944336
Epoch 3	Top1 Train accuracy 24.65544891357422	Top1 Test accuracy: 22.399259567260742	Top5 test acc: 72.59457397460938
Epoch 4	Top1 Train accuracy 24.971956253051758	Top1 Test accuracy: 21.96751594543457	Top5 test acc: 72.2245101928711
Epoch 5	Top1 Train accuracy 25.590946197509766	Top1 Test accuracy: 21.77220344543457	Top5 test acc: 71.81332397460938
Epoch 6	Top1 Train accuracy 25.799280166625977	Top1 Test accuracy: 22.327302932739258	Top5 test acc: 72.09087371826172
Epoch 7	Top1 Train accuracy 26.34415054321289	Top1 Test accuracy: 23.005756378173828	Top5 test acc: 71.64884948730469
Epoch 8	Top1 Train accuracy 25.98357391357422	Top1 Test acc